In [2]:
import librosa
from librosa import display
import os
import glob 
import numpy as np
import time

In [5]:
path = 'data/'
lst = []
i = -2
start_time = time.time()

for subdir, dirs, files in os.walk(path):
  i=i+1
  print(subdir)
  print(i)
  for file in files:

        X, sample_rate = librosa.load(os.path.join(subdir,file), res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_fft=4096, hop_length=256, n_mfcc=40).T,axis=0) 
        arr = mfccs, i
        lst.append(arr) #Here we append the MFCCs to our list.

print("--- Datos cargados correctamente. Tiempo procesado: %s segundos ---" % (time.time() - start_time))

data/
-1
data/SAD
0
data/HAPPY
1
data/NEUTRAL
2
--- Datos cargados correctamente. Tiempo procesado: 268.94951009750366 segundos ---


In [6]:
X, y = zip(*lst)
X = np.asarray(X)
y = np.asarray(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

x_traincnn = np.expand_dims(X_train, axis=2) 
x_testcnn = np.expand_dims(X_test, axis=2)

In [7]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

2023-05-17 08:23:39.641241: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-17 08:23:39.952590: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-17 08:23:39.953766: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 08:23:41.298735: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
model = keras.Sequential([

        # input layer
        keras.layers.Flatten(input_shape=(40, 1)),

        # 1st dense layer
        keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.1)),
        keras.layers.Dropout(0.5),

        # 2nd dense layer
        keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dropout(0.5),

        # 3rd dense layer
        keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),

        # output layer
        keras.layers.Dense(10, activation='softmax')
    ])

    # compile model
optimiser = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [27]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=124, epochs=900, validation_data=(x_testcnn, y_test))

Epoch 1/900
24/24 [==============================] - 0s 9ms/step - loss: 1.6831 - accuracy: 0.4759 - val_loss: 1.6589 - val_accuracy: 0.5854
Epoch 2/900
24/24 [==============================] - 0s 8ms/step - loss: 1.6838 - accuracy: 0.4766 - val_loss: 1.6580 - val_accuracy: 0.5854
Epoch 3/900
24/24 [==============================] - 0s 8ms/step - loss: 1.6771 - accuracy: 0.4790 - val_loss: 1.6567 - val_accuracy: 0.5826
Epoch 4/900
24/24 [==============================] - 0s 7ms/step - loss: 1.6859 - accuracy: 0.4762 - val_loss: 1.6557 - val_accuracy: 0.5826
Epoch 5/900
24/24 [==============================] - 0s 10ms/step - loss: 1.6799 - accuracy: 0.4879 - val_loss: 1.6541 - val_accuracy: 0.5840
Epoch 6/900
24/24 [==============================] - 0s 8ms/step - loss: 1.6652 - accuracy: 0.4921 - val_loss: 1.6519 - val_accuracy: 0.5826
Epoch 7/900
24/24 [==============================] - 0s 8ms/step - loss: 1.6844 - accuracy: 0.4659 - val_loss: 1.6501 - val_accuracy: 0.5826
Epoch 8/900


Epoch 59/900
24/24 [==============================] - 0s 7ms/step - loss: 1.5925 - accuracy: 0.4900 - val_loss: 1.5734 - val_accuracy: 0.5799
Epoch 60/900
24/24 [==============================] - 0s 7ms/step - loss: 1.5943 - accuracy: 0.4910 - val_loss: 1.5718 - val_accuracy: 0.5813
Epoch 61/900
24/24 [==============================] - 0s 8ms/step - loss: 1.5905 - accuracy: 0.4945 - val_loss: 1.5712 - val_accuracy: 0.5799
Epoch 62/900
24/24 [==============================] - 0s 8ms/step - loss: 1.5921 - accuracy: 0.4962 - val_loss: 1.5700 - val_accuracy: 0.5799
Epoch 63/900
24/24 [==============================] - 0s 7ms/step - loss: 1.5941 - accuracy: 0.4886 - val_loss: 1.5693 - val_accuracy: 0.5813
Epoch 64/900
24/24 [==============================] - 0s 8ms/step - loss: 1.6017 - accuracy: 0.4869 - val_loss: 1.5683 - val_accuracy: 0.5826
Epoch 65/900
24/24 [==============================] - 0s 8ms/step - loss: 1.5851 - accuracy: 0.4941 - val_loss: 1.5669 - val_accuracy: 0.5799
Epoch 

24/24 [==============================] - 0s 8ms/step - loss: 1.4628 - accuracy: 0.5000 - val_loss: 1.4430 - val_accuracy: 0.5895
Epoch 174/900
24/24 [==============================] - 0s 7ms/step - loss: 1.4610 - accuracy: 0.5086 - val_loss: 1.4420 - val_accuracy: 0.5895
Epoch 175/900
24/24 [==============================] - 0s 7ms/step - loss: 1.4643 - accuracy: 0.5014 - val_loss: 1.4417 - val_accuracy: 0.5854
Epoch 176/900
24/24 [==============================] - 0s 8ms/step - loss: 1.4620 - accuracy: 0.5017 - val_loss: 1.4408 - val_accuracy: 0.5854
Epoch 177/900
24/24 [==============================] - 0s 8ms/step - loss: 1.4520 - accuracy: 0.5052 - val_loss: 1.4399 - val_accuracy: 0.5771
Epoch 178/900
24/24 [==============================] - 0s 8ms/step - loss: 1.4544 - accuracy: 0.5189 - val_loss: 1.4385 - val_accuracy: 0.5840
Epoch 179/900
24/24 [==============================] - 0s 7ms/step - loss: 1.4598 - accuracy: 0.5034 - val_loss: 1.4371 - val_accuracy: 0.5854
Epoch 180/900

24/24 [==============================] - 0s 9ms/step - loss: 1.3532 - accuracy: 0.5337 - val_loss: 1.3417 - val_accuracy: 0.5909
Epoch 288/900
24/24 [==============================] - 0s 8ms/step - loss: 1.3459 - accuracy: 0.5255 - val_loss: 1.3407 - val_accuracy: 0.5882
Epoch 289/900
24/24 [==============================] - 0s 7ms/step - loss: 1.3508 - accuracy: 0.5244 - val_loss: 1.3392 - val_accuracy: 0.5909
Epoch 290/900
24/24 [==============================] - 0s 8ms/step - loss: 1.3610 - accuracy: 0.5079 - val_loss: 1.3390 - val_accuracy: 0.5882
Epoch 291/900
24/24 [==============================] - 0s 8ms/step - loss: 1.3492 - accuracy: 0.5165 - val_loss: 1.3383 - val_accuracy: 0.5868
Epoch 292/900
24/24 [==============================] - 0s 8ms/step - loss: 1.3563 - accuracy: 0.5162 - val_loss: 1.3377 - val_accuracy: 0.5923
Epoch 293/900
24/24 [==============================] - 0s 8ms/step - loss: 1.3500 - accuracy: 0.5234 - val_loss: 1.3372 - val_accuracy: 0.5923
Epoch 294/900

24/24 [==============================] - 0s 8ms/step - loss: 1.2819 - accuracy: 0.5310 - val_loss: 1.2628 - val_accuracy: 0.5978
Epoch 402/900
24/24 [==============================] - 0s 7ms/step - loss: 1.2749 - accuracy: 0.5337 - val_loss: 1.2622 - val_accuracy: 0.5950
Epoch 403/900
24/24 [==============================] - 0s 7ms/step - loss: 1.2711 - accuracy: 0.5379 - val_loss: 1.2615 - val_accuracy: 0.5964
Epoch 404/900
24/24 [==============================] - 0s 7ms/step - loss: 1.2718 - accuracy: 0.5331 - val_loss: 1.2609 - val_accuracy: 0.5992
Epoch 405/900
24/24 [==============================] - 0s 7ms/step - loss: 1.2749 - accuracy: 0.5334 - val_loss: 1.2607 - val_accuracy: 0.5992
Epoch 406/900
24/24 [==============================] - 0s 8ms/step - loss: 1.2693 - accuracy: 0.5441 - val_loss: 1.2596 - val_accuracy: 0.5992
Epoch 407/900
24/24 [==============================] - 0s 8ms/step - loss: 1.2681 - accuracy: 0.5296 - val_loss: 1.2588 - val_accuracy: 0.5978
Epoch 408/900

24/24 [==============================] - 0s 8ms/step - loss: 1.2105 - accuracy: 0.5410 - val_loss: 1.1959 - val_accuracy: 0.6006
Epoch 516/900
24/24 [==============================] - 0s 7ms/step - loss: 1.2045 - accuracy: 0.5472 - val_loss: 1.1954 - val_accuracy: 0.5992
Epoch 517/900
24/24 [==============================] - 0s 8ms/step - loss: 1.2137 - accuracy: 0.5410 - val_loss: 1.1952 - val_accuracy: 0.6006
Epoch 518/900
24/24 [==============================] - 0s 7ms/step - loss: 1.2141 - accuracy: 0.5403 - val_loss: 1.1950 - val_accuracy: 0.5992
Epoch 519/900
24/24 [==============================] - 0s 7ms/step - loss: 1.2105 - accuracy: 0.5324 - val_loss: 1.1945 - val_accuracy: 0.5978
Epoch 520/900
24/24 [==============================] - 0s 8ms/step - loss: 1.2117 - accuracy: 0.5413 - val_loss: 1.1943 - val_accuracy: 0.6006
Epoch 521/900
24/24 [==============================] - 0s 7ms/step - loss: 1.2123 - accuracy: 0.5448 - val_loss: 1.1947 - val_accuracy: 0.5978
Epoch 522/900

24/24 [==============================] - 0s 8ms/step - loss: 1.1595 - accuracy: 0.5537 - val_loss: 1.1396 - val_accuracy: 0.6033
Epoch 630/900
24/24 [==============================] - 0s 8ms/step - loss: 1.1618 - accuracy: 0.5623 - val_loss: 1.1382 - val_accuracy: 0.6006
Epoch 631/900
24/24 [==============================] - 0s 7ms/step - loss: 1.1526 - accuracy: 0.5541 - val_loss: 1.1381 - val_accuracy: 0.6033
Epoch 632/900
24/24 [==============================] - 0s 7ms/step - loss: 1.1601 - accuracy: 0.5441 - val_loss: 1.1379 - val_accuracy: 0.6019
Epoch 633/900
24/24 [==============================] - 0s 7ms/step - loss: 1.1627 - accuracy: 0.5513 - val_loss: 1.1371 - val_accuracy: 0.6019
Epoch 634/900
24/24 [==============================] - 0s 7ms/step - loss: 1.1622 - accuracy: 0.5517 - val_loss: 1.1367 - val_accuracy: 0.6033
Epoch 635/900
24/24 [==============================] - 0s 7ms/step - loss: 1.1573 - accuracy: 0.5541 - val_loss: 1.1361 - val_accuracy: 0.6006
Epoch 636/900

24/24 [==============================] - 0s 7ms/step - loss: 1.1196 - accuracy: 0.5579 - val_loss: 1.0882 - val_accuracy: 0.6116
Epoch 744/900
24/24 [==============================] - 0s 8ms/step - loss: 1.1197 - accuracy: 0.5592 - val_loss: 1.0881 - val_accuracy: 0.6116
Epoch 745/900
24/24 [==============================] - 0s 7ms/step - loss: 1.1054 - accuracy: 0.5613 - val_loss: 1.0872 - val_accuracy: 0.6102
Epoch 746/900
24/24 [==============================] - 0s 8ms/step - loss: 1.1179 - accuracy: 0.5572 - val_loss: 1.0850 - val_accuracy: 0.6061
Epoch 747/900
24/24 [==============================] - 0s 8ms/step - loss: 1.1131 - accuracy: 0.5596 - val_loss: 1.0850 - val_accuracy: 0.6088
Epoch 748/900
24/24 [==============================] - 0s 7ms/step - loss: 1.1096 - accuracy: 0.5592 - val_loss: 1.0828 - val_accuracy: 0.6074
Epoch 749/900
24/24 [==============================] - 0s 7ms/step - loss: 1.1135 - accuracy: 0.5610 - val_loss: 1.0833 - val_accuracy: 0.6143
Epoch 750/900

24/24 [==============================] - 0s 8ms/step - loss: 1.0673 - accuracy: 0.5699 - val_loss: 1.0277 - val_accuracy: 0.6143
Epoch 858/900
24/24 [==============================] - 0s 8ms/step - loss: 1.0659 - accuracy: 0.5764 - val_loss: 1.0258 - val_accuracy: 0.6116
Epoch 859/900
24/24 [==============================] - 0s 7ms/step - loss: 1.0588 - accuracy: 0.5802 - val_loss: 1.0245 - val_accuracy: 0.6116
Epoch 860/900
24/24 [==============================] - 0s 7ms/step - loss: 1.0724 - accuracy: 0.5730 - val_loss: 1.0262 - val_accuracy: 0.6129
Epoch 861/900
24/24 [==============================] - 0s 8ms/step - loss: 1.0671 - accuracy: 0.5764 - val_loss: 1.0249 - val_accuracy: 0.6116
Epoch 862/900
24/24 [==============================] - 0s 7ms/step - loss: 1.0649 - accuracy: 0.5837 - val_loss: 1.0226 - val_accuracy: 0.6102
Epoch 863/900
24/24 [==============================] - 0s 8ms/step - loss: 1.0631 - accuracy: 0.5823 - val_loss: 1.0230 - val_accuracy: 0.6143
Epoch 864/900

In [28]:
import matplotlib as plt
import numpy as np

In [29]:
from sklearn.metrics import classification_report

predictions = np.argmax(model.predict(x_testcnn),axis=-1)
y_test = y_test.astype("int")
report = classification_report(y_test, predictions)
print(report)

23/23 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.66      0.79      0.72       280
           1       0.74      0.66      0.69       249
           2       0.41      0.36      0.38       197

    accuracy                           0.63       726
   macro avg       0.60      0.60      0.60       726
weighted avg       0.62      0.63      0.62       726



In [30]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, predictions)
print (matrix)

[[222  12  46]
 [ 32 164  53]
 [ 80  47  70]]


In [31]:
model_name = 'EmotionClassificationModel.h5'
save_dir = 'trainedModel/'
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at trainedModel/EmotionClassificationModel.h5 
